In [1]:
from pyspark.sql import SparkSession
from delta import *
from minio import Minio
import pyspark.sql.functions as F
import os
import pandas as pd

In [2]:
#CONFIGURAÇÕES SPARK
spark = (
    SparkSession.builder.master("local[*]").appName("csvToParquet")
    #HADOOP
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    #DELTA
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.jars", "jars/aws-java-sdk-bundle-1.11.1026.jar, \
            jars/hadoop-aws-3.2.0.jar")
    .getOrCreate()
)


24/09/19 04:44:32 WARN Utils: Your hostname, DESKTOP-CV9TLA8 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/19 04:44:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hacs/.cache/pypoetry/virtualenvs/pyspark-delta-gPM1PAZH-py3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hacs/.ivy2/cache
The jars for the packages stored in: /home/hacs/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4c8a609c-1b38-4f85-b630-62a02189a9de;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 170ms :: artifacts dl 6ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0

In [3]:
#CONFIGURAÇÕES CLIENT DO MINIO
client=Minio("localhost:9000",
             access_key="minioadmin",
             secret_key="minioadmin",
             secure=False
            )


In [4]:
bucket_name = 'raw'
objects=client.list_objects(bucket_name, prefix='CAPES/', recursive=True)
csvs = []
for object in objects:
    csvs.append(object.object_name)

#LISTANDO OS DATAFRAMES PRESENTES NO BUCKET RAW DO MinIO
count = 0
for csv in csvs:
    print(f'{count} - {csv}')
    count+=1

0 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022 Bibliográfica Subtipo Livro.csv
1 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Artigo em Jornal ou Revista.csv
2 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Outro.csv
3 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Aplicativo.csv
4 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Produto.csv
5 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnic

In [10]:
#FUNÇÃO DE FILTRO PELA COLUNA ESPECIFICA USANDO A KEY ESPECIFICA
#OS PARAMETROS SAO: 
#NOME DO ARQUIVO NA DATABASE = name_database
#A DATABASE EM SI = database
#O CLIENT DO MINIO = client
#A COLUNA ESPECIFICA = column_key
#A KEY ESPECIFICA = key
def filterByColumn(name_database, database, client, column_key, key):
    #PARA CADA COLUNA DA DATABASE, EU PERCORRO ATE ENCONTRAR A QUE EU QUERO, SENAO NAO FAZ NADA
    for column_actual in database.columns:
        if column_actual == column_key:
            #SE EU ACHO ENTAO FILTRO USANDO OS PARAMETROS PASSADOS
            database = database.filter(F.col(column_key).like(key))
            #APOS USO O PANDAS PARA CONVERTE-LO EM CSV NOVAMENTE PARA CRIAR UM ARQUIVO TEMPORARIO PARA CONSEGUIR USAR O MINIO
            database.toPandas().to_csv('temp.csv', sep=';', index=False)
            client.fput_object('raw',f"{name_database}", 'temp.csv')
            #REMOVO O TEMPORARIO E FINALIZO A FUNÇÃO POIS JA FOI FEITO AS ALTERAÇÕES
            os.remove('temp.csv')
            break

#A FUNÇÃO MODIFICA DIRETAMENTE O BANCO DE DADOS, SE QUER SALVAR AS ALTERAÇÕES EM OUTRO LUGAR ENTÃO APENAS MODIFIQUE A LINHA
#client.fput_object('raw',f"{name_database}", 'temp.csv'), COLOCANDO O LUGAR ESPECIFICO A SER SALVO, EXEMPLO SE QUER COLOCAR NUMA
#PASTA CHAMADA UFMA, ENTAO: client.fput_object('raw',f"UFMA/{name_database}", 'temp.csv')
#OS ARQUIVOS SEMPRE SERAO SALVOS EM UMA SUBPASTA CHAMADA CAPES/ POIS OS MESMOS JA POSSUEM ELA NO NOME

In [6]:
#NOS FILTROS, FORAM USADOS QUATRO COLUNAS POIS VARIOS ARQUIVOS POSSUIAM NOMENCLATURAS DIFERENTES PARA A AREA DE INTERESSE
for csv_number in range(0, len(csvs)):
    database = spark.read.format('csv').option('header', 'true').option('sep', ';')\
        .load(f's3a://{bucket_name}/{csvs[csv_number]}')
    filterByColumn(csvs[csv_number], database, client, "SG_ENTIDADE_ENSINO", "%UFMA%")
    print(f"{csvs[csv_number]} OK")

24/09/19 04:44:36 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/09/19 04:44:39 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022 Bibliográfica Subtipo Livro.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Artigo em Jornal ou Revista.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Outro.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Aplicativo.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Produto.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo

In [7]:
for csv_number in range(0, len(csvs)):
    database = spark.read.format('csv').option('header', 'true').option('sep', ';')\
        .load(f's3a://{bucket_name}/{csvs[csv_number]}')
    filterByColumn(csvs[csv_number], database, client, "NM_AREA_CONHECIMENTO", "%COMPUTA%")
    print(f"{csvs[csv_number]} OK")

CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022 Bibliográfica Subtipo Livro.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Artigo em Jornal ou Revista.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Outro.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Aplicativo.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Produto.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo

In [8]:
for csv_number in range(0, len(csvs)):
    database = spark.read.format('csv').option('header', 'true').option('sep', ';')\
        .load(f's3a://{bucket_name}/{csvs[csv_number]}')
    filterByColumn(csvs[csv_number], database, client, "NM_AREA_CONCENTRACAO", "%COMPUTA%")
    print(f"{csvs[csv_number]} OK")

CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022 Bibliográfica Subtipo Livro.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Artigo em Jornal ou Revista.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Outro.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Aplicativo.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Produto.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo

In [9]:
for csv_number in range(0, len(csvs)):
    database = spark.read.format('csv').option('header', 'true').option('sep', ';')\
        .load(f's3a://{bucket_name}/{csvs[csv_number]}')
    filterByColumn(csvs[csv_number], database, client, "NM_AREA_AVALIACAO", "%COMPUTA%")
    print(f"{csvs[csv_number]} OK")

CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022 Bibliográfica Subtipo Livro.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Artigo em Jornal ou Revista.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Outro.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Aplicativo.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Produto.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo